In [1]:
import os

from time import sleep
import config
import config_media_costs
from config_search_funcs import get_subbrand_id_str
from media_costs import get_media_costs_report, get_mon_num_from_date, update_media_cost
from create_dicts_adex import create_adex_tables, download_adex_default_dicts, update_media_ads_dict, update_ad_dict_items, update_another_dicts, update_list_dicts, create_adex_views
from google_connector import append_ads_to_google, query_for_google


# subbrand_id_str = get_subbrand_id_str(mon_num='360')
# special_ad_filter = f'{config_media_costs.main_filter_str} AND t10.sbid in ({subbrand_id_str})'

start_date = '2023-01-01' # если дата начала НЕ указана, то start_date = Сегодня минус 3 дня минус 17 дней
end_date = '2025-05-02' #- ВКЛЮЧИТЕЛЬНО / если дата окончания НЕ указана, то end_date = Сегодня минус 3 дня

flag='first' #first / regular

Все ок. Подключились!
Загрузка завершена успешно


In [2]:
# создаем пустые таблицы Фактов и Справочники
# Заполняем справоники по умолчанию данными config_media_costs.adex_defult_dicts
# Company / Ad_type / EState / 
if flag=='first':
    # создаем все пустые таблицы
    create_adex_tables()
    sleep(10)
    # наполняем данными справочники, которые НЕ обновляются
    download_adex_default_dicts()
    sleep(10)
    # создаем Представления
    create_adex_views()

# забираем статистику по каждому Медиа за указанный период
for key, value in config.media_type_dict.items():
    get_media_costs_report(start_date=start_date, end_date=end_date, media_type=key,  flag=flag)


Все ок. Подключились!
Пустая таблица adex_distribution_dict успешно создана в БД mediascope_x5
Все ок. Подключились!
Пустая таблица adex_estate_dict успешно создана в БД mediascope_x5
Все ок. Подключились!
Пустая таблица adex_regions_dict успешно создана в БД mediascope_x5
Все ок. Подключились!
Пустая таблица media_tv_costs успешно создана в БД mediascope_x5
Все ок. Подключились!
Пустая таблица adex_company_dict_tv успешно создана в БД mediascope_x5
Все ок. Подключились!
Пустая таблица adex_ad_type_dict_tv успешно создана в БД mediascope_x5
Все ок. Подключились!
Пустая таблица adex_audio_slogan_dict_tv успешно создана в БД mediascope_x5
Все ок. Подключились!
Пустая таблица adex_video_slogan_dict_tv успешно создана в БД mediascope_x5
Все ок. Подключились!
Пустая таблица adex_ad_dict_list_tv успешно создана в БД mediascope_x5
Все ок. Подключились!
Пустая таблица media_radio_costs успешно создана в БД mediascope_x5
Все ок. Подключились!
Пустая таблица adex_company_dict_radio успешно созда

In [3]:
for media_type, media_type_long in config.media_type_dict.items():
    # здесь загружаем недостающие ИД
    update_media_ads_dict(media_type)
    # здесь обновляем кастомные поля(флаг чистки, категория и тд) в справочнике объявлений
    update_ad_dict_items(media_type)
    update_another_dicts(media_type)
    update_list_dicts(media_type)

Все ок. Подключились!
Загрузка завершена успешно
Все ок. Подключились!
Загрузка завершена успешно
Все ок. Подключились!
Загрузка завершена успешно
Все ок. Подключились!
Скрипт запущен 2025-05-29 22:42:37.291183
Данные добавлены в БД: mediascope_x5, таблица: adex_ad_dict_list_tv
Скрипт отработал 2025-05-29 22:42:40.421141
Время выполнения задачи: 0:00:03.129958
Загрузка завершена. Данные успешно добавлены в БД: mediascope_x5
##################################################

Все ок. Подключились!
Загрузка завершена успешно
Все ок. Подключились!
Таблица: adex_ad_dict_list_tv успешно удалена в БД: mediascope_x5
##########
Все ок. Подключились!
Пустая таблица adex_ad_dict_list_tv успешно создана в БД mediascope_x5
Все ок. Подключились!
Скрипт запущен 2025-05-29 22:42:53.906293
Данные добавлены в БД: mediascope_x5, таблица: adex_ad_dict_list_tv
Скрипт отработал 2025-05-29 22:42:56.468112
Время выполнения задачи: 0:00:02.561819
Загрузка завершена. Данные успешно добавлены в БД: mediascope_x

In [4]:
# загружаем в гугл докс Чистка список объяалений с флагом=2
query = query_for_google(report='media')
worksheet = config_media_costs.google_new_ads_media_invest['worksheet']

append_ads_to_google(query=query, worksheet=worksheet)

Все ок. Подключились!
Загрузка завершена успешно
Connection established successfully...
DataFrame exported successfully...


In [5]:
# # Может возникнуть необходимость перезаписать кастомные поля в таблицах расходов
# # например - пересмотрели размер дисконта / изменили правила опеределения одной из кастомных колонок и тд.
# # В этом случае забираем из нашей БД из каждой таблицы расходов колонки, которые мы взяли из Медиаскопа
# # И пропускаем по тем же шагам обработки, которые применялись в первоначальной загрузке
# # По итогу - удаляем существующую таблицу в БД и на ее место записываем таблицу с новуми данными
# for media_type in config.media_type_dict.keys():
#     update_media_cost(media_type)